<a href="https://colab.research.google.com/github/OwlMa/CS634/blob/master/CS634_titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
from google.colab import files
upload=files.upload()

Saving test.csv to test (2).csv
Saving train.csv to train (2).csv


In [66]:
import os
import tarfile
import pandas as pd
import numpy as np
from pandas import Series,DataFrame

import io
titanic_train = pd.read_csv('train.csv')
titanic_test = pd.read_csv('test.csv')
titanic_test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [67]:
titanic=titanic_train.append(titanic_test,ignore_index=True)
titanic.shape

(1309, 12)

In [68]:
titanic.info()
# we can see the missing data in the train set
# Age, Cabin and Embarked are three missing data, so we need to fill these three kinds of data

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 12 columns):
Age            1046 non-null float64
Cabin          295 non-null object
Embarked       1307 non-null object
Fare           1308 non-null float64
Name           1309 non-null object
Parch          1309 non-null int64
PassengerId    1309 non-null int64
Pclass         1309 non-null int64
Sex            1309 non-null object
SibSp          1309 non-null int64
Survived       891 non-null float64
Ticket         1309 non-null object
dtypes: float64(3), int64(4), object(5)
memory usage: 122.8+ KB


In [69]:
titanic['Age']=titanic['Age'].fillna(titanic['Age'].mean())
titanic=titanic.drop(["Cabin"],axis=1)
titanic=titanic.drop(['Name'],axis=1)
titanic['Embarked']=titanic['Embarked'].fillna('S')
titanic['Fare']=titanic['Fare'].fillna(titanic['Fare'].mean())
titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 10 columns):
Age            1309 non-null float64
Embarked       1309 non-null object
Fare           1309 non-null float64
Parch          1309 non-null int64
PassengerId    1309 non-null int64
Pclass         1309 non-null int64
Sex            1309 non-null object
SibSp          1309 non-null int64
Survived       891 non-null float64
Ticket         1309 non-null object
dtypes: float64(3), int64(4), object(3)
memory usage: 102.3+ KB


In [70]:
#change the text feature to numerical feature 
titanic['Sex'].value_counts()
sexDict={'male':1,'female':0}
titanic['Sex']=titanic['Sex'].map(sexDict)
titanic.head()

,Age,Embarked,Fare,Parch,PassengerId,Pclass,Sex,SibSp,Survived,Ticket
0,22.0,S,7.2500,0,1,3,1,1,0.0,A/5 21171
1,38.0,C,71.2833,0,2,1,0,1,1.0,PC 17599
2,26.0,S,7.9250,0,3,3,0,0,1.0,STON/O2. 3101282
3,35.0,S,53.1000,0,4,1,0,1,1.0,113803
4,35.0,S,8.0500,0,5,3,1,0,0.0,373450


In [71]:
embarked=pd.DataFrame()
embarked=pd.get_dummies(titanic['Embarked'],prefix='Embarked')
embarked.head()

,Embarked_C,Embarked_Q,Embarked_S
0,0,0,1
1,1,0,0
2,0,0,1
3,0,0,1
4,0,0,1


In [72]:
pclass=pd.DataFrame()
pclass=pd.get_dummies(titanic['Pclass'],prefix='Pclass')
pclass.head()

,Pclass_1,Pclass_2,Pclass_3
0,0,0,1
1,1,0,0
2,0,0,1
3,1,0,0
4,0,0,1


In [73]:
titanic=pd.concat([titanic,pclass],axis=1)
titanic.drop('Pclass',axis=1,inplace=True)
titanic.head()

,Age,Embarked,Fare,Parch,PassengerId,Sex,SibSp,Survived,Ticket,Pclass_1,Pclass_2,Pclass_3
0,22.0,S,7.2500,0,1,1,1,0.0,A/5 21171,0,0,1
1,38.0,C,71.2833,0,2,0,1,1.0,PC 17599,1,0,0
2,26.0,S,7.9250,0,3,0,0,1.0,STON/O2. 3101282,0,0,1
3,35.0,S,53.1000,0,4,0,1,1.0,113803,1,0,0
4,35.0,S,8.0500,0,5,1,0,0.0,373450,0,0,1


In [74]:
titanic=pd.concat([titanic,embarkedDF],axis=1)
titanic.drop('Embarked',axis=1,inplace=True)
titanic.head()

,Age,Fare,Parch,PassengerId,Sex,SibSp,Survived,Ticket,Pclass_1,Pclass_2,Pclass_3,Embarked_C,Embarked_Q,Embarked_S
0,22.0,7.2500,0,1,1,1,0.0,A/5 21171,0,0,1,0,0,1
1,38.0,71.2833,0,2,0,1,1.0,PC 17599,1,0,0,1,0,0
2,26.0,7.9250,0,3,0,0,1.0,STON/O2. 3101282,0,0,1,0,0,1
3,35.0,53.1000,0,4,0,1,1.0,113803,1,0,0,0,0,1
4,35.0,8.0500,0,5,1,0,0.0,373450,0,0,1,0,0,1


In [75]:
#Because the GD is sensitive to the diffrent features,we need to scale the large scale features 'Age' and 'Fare'.
from sklearn.preprocessing import scale
age_scaled=scale(titanic['Age'])
titanic['Age']=age_scaled
titanic.head()

,Age,Fare,Parch,PassengerId,Sex,SibSp,Survived,Ticket,Pclass_1,Pclass_2,Pclass_3,Embarked_C,Embarked_Q,Embarked_S
0,-0.611972,7.2500,0,1,1,1,0.0,A/5 21171,0,0,1,0,0,1
1,0.630431,71.2833,0,2,0,1,1.0,PC 17599,1,0,0,1,0,0
2,-0.301371,7.9250,0,3,0,0,1.0,STON/O2. 3101282,0,0,1,0,0,1
3,0.397481,53.1000,0,4,0,1,1.0,113803,1,0,0,0,0,1
4,0.397481,8.0500,0,5,1,0,0.0,373450,0,0,1,0,0,1


In [76]:
fare_scaled=scale(titanic['Fare'])
titanic['Fare']=fare_scaled
titanic.head()

,Age,Fare,Parch,PassengerId,Sex,SibSp,Survived,Ticket,Pclass_1,Pclass_2,Pclass_3,Embarked_C,Embarked_Q,Embarked_S
0,-0.611972,-0.503595,0,1,1,1,0.0,A/5 21171,0,0,1,0,0,1
1,0.630431,0.734503,0,2,0,1,1.0,PC 17599,1,0,0,1,0,0
2,-0.301371,-0.490544,0,3,0,0,1.0,STON/O2. 3101282,0,0,1,0,0,1
3,0.397481,0.382925,0,4,0,1,1.0,113803,1,0,0,0,0,1
4,0.397481,-0.488127,0,5,1,0,0.0,373450,0,0,1,0,0,1


In [77]:
#corralation
corr=titanic.corr()
corr['Survived'].sort_values(ascending=False)

Survived       1.000000
Pclass_1       0.285904
Fare           0.257307
Embarked_C     0.168240
Pclass_2       0.093349
Parch          0.081629
Embarked_Q     0.003650
PassengerId   -0.005007
SibSp         -0.035322
Age           -0.070323
Embarked_S    -0.149683
Pclass_3      -0.322308
Sex           -0.543351
Name: Survived, dtype: float64

In [78]:
titanic_x=pd.concat([pclass,embarked,titanic['Age'],titanic['Fare'],titanic['Sex'],titanic['SibSp'],titanic['Parch']],axis=1)
titanic_x.head()

,Pclass_1,Pclass_2,Pclass_3,Embarked_C,Embarked_Q,Embarked_S,Age,Fare,Sex,SibSp,Parch
0,0,0,1,0,0,1,-0.611972,-0.503595,1,1,0
1,1,0,0,1,0,0,0.630431,0.734503,0,1,0
2,0,0,1,0,0,1,-0.301371,-0.490544,0,0,0
3,1,0,0,0,0,1,0.397481,0.382925,0,1,0
4,0,0,1,0,0,1,0.397481,-0.488127,1,0,0


In [79]:
row = 891
whole_X = titanic_x.loc[0:row-1,:] 
whole_y = titanic.loc[0:row-1,'Survived']
pred_X = titanic_x.loc[row:,:]
pred_X

,Pclass_1,Pclass_2,Pclass_3,Embarked_C,Embarked_Q,Embarked_S,Age,Fare,Sex,SibSp,Parch
891,0,0,1,0,1,0,0.358655,-0.492396,1,0,0
892,0,0,1,0,0,1,1.329283,-0.508429,0,1,0
893,0,1,0,0,1,0,2.494035,-0.456465,1,0,0
894,0,0,1,0,0,1,-0.223721,-0.476284,1,0,0
895,0,0,1,0,0,1,-0.611972,-0.406194,0,1,1
896,0,0,1,0,0,1,-1.233173,-0.465408,1,0,0
897,0,0,1,0,1,0,0.009230,-0.496263,0,0,0
898,0,1,0,0,0,1,-0.301371,-0.083054,1,1,1
899,0,0,1,1,0,0,-0.922572,-0.503997,0,0,0
900,0,0,1,0,0,1,-0.689622,-0.176830,1,2,0


In [80]:
from sklearn.model_selection import train_test_split
train_X,test_X,train_y,test_y = train_test_split(whole_X,whole_y,train_size=0.8)
whole_y.head()

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


0    0.0
1    1.0
2    1.0
3    1.0
4    0.0
Name: Survived, dtype: float64

In [81]:
from sklearn.ensemble import GradientBoostingClassifier
modelGD = GradientBoostingClassifier()
modelGD.fit(train_X,train_y)
predict=modelGD.predict(test_X)
modelGD.score(test_X,test_y)

0.8379888268156425

In [82]:
#MSE
from sklearn import metrics
metrics.mean_squared_error(test_y, predict)

0.16201117318435754

In [83]:
pred_y = modelGD.predict(pred_X)
pred_y = pred_y.astype(int)
passenger_id = titanic.loc[row:,'PassengerId']
predict = pd.DataFrame({'PassengerId':passenger_id, 'Survived':pred_y})
predict

,PassengerId,Survived
891,892,0
892,893,0
893,894,0
894,895,0
895,896,0
896,897,0
897,898,1
898,899,0
899,900,1
900,901,0


In [0]:
predict.to_csv('submission.csv',index=False)

In [0]:
files.download('submission.csv')